## Currents and fields

In this notebook we use Python to attack electrostatics problems that would be difficult or impossible to do in *closed form*. 

$$B = q \frac{\mu_0}{4\pi} \frac{\vec v \times \hat r}{r^2} $$

$B$ is the magnetic field as a point a distance $r$ from a charge $q$ travelling with velocity $v$.  $\hat r$ is the unit vector from the point to the charge. $\times$ is the vector cross product.

Ignoring the constants, we can re-write this equation as
$$B = \frac{\vec I \times \hat r}{r^2}$$
where $\vec I = q \vec v$ is the current vector.

So if our current was running in a wire, our magnetic field at any given point would be given by an integral (over a curve)
 
 $$B = \int \frac{d\vec I \times \vec r}{r^3}$$

similarly if the current was running through a solid, the above integral would be over a 3-dimensional region.  To make this a little more explicit, if we wish to evaluate the vector field $B$ at a point $\vec p$, we get the expression

$$B(\vec p) = \int \frac{d \vec I \times (\vec x - \vec p)}{|\vec x - \vec p|^3}$$

where $\vec x$ is the variable of integration, and $\vec r = \vec x - \vec p$, and $ \vec I$ is a function of $\vec x$, i.e. $\vec I(\vec x)$.  So in this case the **integrand** is a vector-valued function 
$$F(\vec x, \vec p) = \frac{d \vec I \times (\vec x - \vec p)}{|\vec x - \vec p|^3}$$
i.e. a function of two $3$-dimensional vector variables, or equivalently six $1$-dimensional real variables, and in our integration we integrate with respect to only three of those six variables -- this allows us to have the three remaining variables of $\vec p$ free after integration. 

 * * *
 
Before we start computing-away, notice that we will have to compute vector-valued integrals over 1, 2 and 3-dimensional domains... but not just one or two such integrals.  To get a sense for these vector fields we need to compute such integrals over a wide range of points in $R^3$ if we hope to get a sense for what these vector fields look like.  

As it is more pleasant to work in vector notation, we modify the scipy integrate library to happily work with vector-valued functions, and to iterate over meshed grids of points.  

In [ ]:
## We will use the scipy integrate.nquad library.   It expect scalar-valued 
## functions.  

import numpy as np
from scipy import integrate

print(integrate.quad(np.sin, 0, np.pi) )

This is good, but we will need to perform families of integrals and sometimes over more than 1-dimensional domains. Moreover, the above integrate routine demands that the *input* function (here it is $np.sin$) is real-valued.  We need to integrate vector-valued functions as well.  We begin to correct these defects in the scipy library. 

In [ ]:
## vecint is a function that computes an integral.  f is a vector valued function
## defined on an n-dimensional "interval" I.  I will be a list of 2-element lists, describing
## the endpoints.  f will be a k-tuple of scalar-valued functions that take n arguments. 
## The return value will be a pair, the first element being the approximate integral.  The
## second being an upper bound on the error (a length in R^k). You can pass additional 
## arguments with "args".
def vecint(F, I, *args):
    componentintegrals = [integrate.nquad(f, I, args) for f in F]
    retint = [CI[0] for CI in componentintegrals]
    if (len(retint)==1):
        retint = retint[0]
    reterr = np.sqrt(sum(CI[1]**2 for CI in componentintegrals))
    return retint, reterr
   
## integral of the vector (sin x, cos y) over the 2-dimensional domain [0,pi]x[0,pi]
print(vecint([lambda x,y: np.sin(x), lambda x,y: np.cos(y)], [[0,np.pi],[0,np.pi]] ) )
## integral of sin(x) on [0,pi]
print(vecint([lambda x: np.sin(x)], [[0,np.pi]]))
## integral of (cos x, sin x) on [0,pi]
print(vecint([lambda x: np.cos(x), lambda x: np.sin(x)], [[0,np.pi]] ))

## and we can pass additional arguments. 

## integral of (sin(x)+k)dx on [0,pi]  with k=1
print(vecint( [lambda x,k: np.sin(x)+k], [[0,np.pi]], 1) )
## integral of (sin(x)+a+b)dx with a=b=0
print(vecint( [lambda x,a,b: np.sin(x)+a+b], [[0,np.pi]], 0,0))

## good!

## But not good enough!

This isn't convenient enough as we will need to perform these kinds of integrals over a wide range of x,y,z coordinates in order to plot the vector fields.   To do this we will utilize the numpy **vectorize** routine.  This is a routine that takes arrays of inputs and provides arrays of outputs, i.e. if one has a function
$$f(x)$$
that takes as input a float $x$, the 'vectorized' version of $f$ will takes as input an array
$$[x_1,x_2,\cdots,x_n]$$
and return an output
$$[f(x_1), f(x_2), \cdots, f(x_n)]$$
Unfortunately for us, numpy can only vectorize a function $f$ if its output is a numpy base data type (for example, an *np.float32*).  So first we vectorize scalar-valued integrals. 

In [ ]:
## We want to vectorize the integration process.  The below produces a vectorized 
## 1-dimensional integration, that takes a 1-dimensional grid of values as points of
## evaluation. 
def quad_1vz1(f, I, *args):
    return np.vectorize(lambda n: integrate.quad(f, I[0], I[1], (n,)+args)[0])

## let's demonstrate it. 
def f(x,k):
    return np.sin(x*k)
K = np.mgrid[0:1:6j]
## integral of f(x,k)dx for various values of k.
print(quad_1vz1(f, [0,1] )(K))

## notice we can make f have more variables and pass those additional arguments
def f(x,k,a):
    return a*np.sin(x*k)

## integral of f(x,k,a)dx for various values of k, with a == 1
print(quad_1vz1(f, [0,1], 1)(K))
## integral of f(x,k,a)dx for various values of k, with a == 2
print(quad_1vz1(f, [0,1], 2)(K))

### And with two parameters. 

In [ ]:
## Let's do the same for 1-dimensional integrals with two parameters
def quad_1vz2(f, I, *args):
    return np.vectorize(lambda n,m: integrate.quad(f, I[0], I[1], (n,m)+args)[0])
X,Y=np.mgrid[0:1:5j, 0:1:5j]
def f(t,x,y):
    return np.sin(t*x)+y
## integral of (sin(tx)+y)dt on [0,1] for (x,y) in a 2-dimensional grid of points.
print(quad_1vz2(f, [0,1])(X,Y))

## Good

Let's vectorize multi-dimensional scalar-valued integrals now. 

In [ ]:
## integration over n-dimensional domain, of real valued function (1d output), and
##  with a single vectorization parameter. 
def nquad_1vz1(f, I, *args):
    return np.vectorize(lambda n: integrate.nquad(f, I, (n,)+args)[0])

def f(x,y,k):
    return x**2-y**2+k

K = np.mgrid[0:1:6j]
nquad_1vz1( f, [[-1,1],[-1,1]] )(K)
## looks good!
## these are the integrals of x^2-y^2+k over the box [-1,1]x[-1,1] for various 
## values of k between 0 and 1.

## Integration with 3-extra parameters

Our most basic integration will always require three vectorization parameters as we need vector fields defined in 3-dimensional space.  So we need the analogous **quad_3vz3** and **nquad_3vz3** functions.  We will produce *quad_3vz3* and *nquad_3vz3* from **quad_1vz3** and **nquad_1vz3**, the corresponding vectorization of scalar-valued functions. 

In [ ]:
def quad_1vz3(f, I, *args):
    return np.vectorize(lambda n,m,p: integrate.quad(f, I[0], I[1], (n,m,p)+args)[0])
def nquad_1vz3(f, I, *args):
    return np.vectorize(lambda n,m,p: integrate.nquad(f, I, (n,m,p)+args)[0])
## and let's do a basic test, integration of a 3-variable function with 3 parameters. 
def f(x,y,z,a,b,c):
    return x**2-y**2+z + a*b*c
A,B,C = np.mgrid[0:1:2j,0:1:2j,0:1:2j]
## these are the integrals of x^2-y^2+abc dxdy over [-1,1]x[-1,1]
nquad_1vz3(f, [[-1,1],[-1,1],[-1,1]] )(A,B,C)
## Looks good! 

## Lastly, we need integration calls for vector-valued functions.  

We have all our building-blocks. We need an integration procedure that takes as input a vector-valued function of $\{4,5,6\}$ variables, and performs an integration with respect to $\{1,2,3\}$ of them, returning mgrids $u,v,w$ 

We want to work with vectors in defining our vector fields as computing the x,y,z coordinates might oftentimes be difficult.  Using sympy we can request the Cartesian coordinates after the vector field has been computed. 

In [ ]:
## and now we're ready to create a vector-valued integration function. 
## this takes as input:
##  F : a list/tuple of single-variable functions on [a,b] with 3 additional parameters
##  I : I = [a,b] a 2-element list describing the domain of the primary parameter
##  returns a tuple of functions
def quad_nvz3(F, I, *args):
    retval = []
    for f in F:
        retval.append([quad_1vz3(f, I)(*args)])
    return tuple(retval)

def f(t,a,b,c):
    return t**2 + a*b*c
def g(t,a,b,c):
    return t**3 + a*b*c
A,B,C = np.mgrid[0:1:2j,0:1:2j,0:1:2j]
## this call returns the integral of the vector
## (t^2+abc, t^3+abc)dt over [0,1] with the three extra parameters abc sampled
##  over a grid. 
X = quad_nvz3([f,g], [0,1], A,B,C)

def nquad_nvz3(F, I, *args):
    retval = []
    for f in F:
        retval.append([nquad_1vz3(f, I)(*args)])
    return tuple(retval)

## nquad_nvz3([f,g,h], I, A,B,C)
## returns the integral ov the vector-valued function
## (f(x,y,z,a,b,c),g(x,y,z,a,b,c),h(x,y,z,a,b,c))dxdydz over the interval I
## with free parameters a,b,c chosen over a grid. 

**Problem 1:** Find the magnetic field surrounding a round circle of radius 1.  We assume the current distribution is uniform. 
 
 Imagine the circle as being decomposed into $n$ equal-length segments.  Each has length $l = \frac{2\pi}{n}$.  This implies the current in a segment of length $l$ is approximately $l \vec I$ where $\vec I$ is a choice of current in the interval.  So our integral is:
 
 $$B = \int \frac{d\vec I \times \vec r}{r^3}$$
 
 so if $\vec p$ is our point in $3$-dimensional space, $\vec r = \vec x - \vec p$ with $\vec x$ on the unit circle, giving the integral:
 
 $$B(\vec p) = \int_C \frac{d \vec I \times (\vec x - \vec p)}{|\vec x - \vec p|^3}$$
 
 if we parametrize the circle as $\vec x = (\cos t, \sin t, 0)$ then we can assume (up to some constant) that $d \vec I = d \vec x = (-\sin t, \cos t, 0)dt$ converting the integral to
 
 $$B(\vec p) = \int_0^{2\pi} \frac{(-\sin t, \cos t, 0)\times \left((\cos t, \sin t, 0) - \vec p\right)}{|(\cos t, \sin t, 0)-\vec p|^3}dt$$
 
Now that we have the integrand in terms of the four variables $t, \vec p$ with $\vec p = (x,y,z)$ we can pass it to *quad_nvz3* and plot the vector field.  But first we need to determine the Cartesian coordinates of this vector field.  Rather than compute them by hand, we ask **sympy** to do the work. 


In [ ]:
import sympy as sp
t,x,y,z = sp.symbols('t x y z')
num = sp.Matrix([-sp.sin(t), sp.cos(t), 0]).cross(\
      sp.Matrix([sp.cos(t)-x, sp.sin(t)-y, -z]))
num.simplify()
den = sp.Matrix([sp.cos(t)-x, sp.sin(t)-y, -z])
den = den.dot(den)**(3/2)
Fs = num/den
sp.pprint(Fs)
## our integrand, in closed form. 

In [ ]:
## now let's cast it into an array of callable functions. I suppose using ufuncify
## or lambdify, either should be fine. 
from sympy.utilities.autowrap import ufuncify

F = []
for i in range(3):
    F.append(ufuncify([t,x,y,z], Fs[i,0]) )

## done. 

In [ ]:
## We set up the integral.   First, decide on the lattice of points where we want to "see" the vector
## field B.  We use meshgrid since matplotlib's 3d-vector-field plotting tool, the "quiver" demands
## your plot be in this format.  We similarly have to compute the vector field B = (u,v,w) along this
## grid. 

## hmm, not awesome.  We should add the conductor.  And maybe put in streamlines? 
## also consider using plotly?  vispy would be overkill and code generation might 
## be painful. 

x, y, z = np.mgrid[-1.5:1.5:8j, -1.5:1.5:8j, -0.5:0.5:6j]

u, v, w = quad_nvz3(F, [0, 2*np.pi], x,y,z )

## distance to circle
#dtc = np.sqrt((np.sqrt(x**2+y**2) - 1.0)**2 + z**2)

In [ ]:
import matplotlib as mpl
%matplotlib nbagg
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

#import itertools as it

mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
ax = fig.gca(projection='3d')

T = np.arange(0.0, 2*np.pi, 0.01)
ax.plot(np.cos(T), np.sin(T), np.zeros_like(T), 'r')
q = ax.quiver(x, y, z, u, v, w, length=0.1, pivot='middle', cmap='Blues', linestyles='solid')

q.set_array(np.random.rand(np.prod(x.shape)))
plt.show()

## quiver call details available in two locations.  The 3d plotting API
## http://matplotlib.org/mpl_toolkits/mplot3d/api.html
## and the linecollection API
## http://matplotlib.org/api/collections_api.html#matplotlib.collections.LineCollection

ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')

plt.show()

In [ ]:
## hmm, it does not look like matplotlib lets us set the arrow colors individually
#colors=np.empty(x.shape+(4,), dtype=np.float)
#print(colors.shape)
#for vi,vj,vk in it.product(range(colors.shape[0]), range(colors.shape[1]), range(colors.shape[2])):
#    colors[vi,vj,vk,0] = 0.9 if (vi+vj+vk%2==0) else 0.0
#    colors[vi,vj,vk,1] = 0.2 if (vi+vj+vk%2==0) else 0.4
#    colors[vi,vj,vk,2] = 0.0 if (vi+vj+vk%2==0) else 0.6
#    colors[vi,vj,vk,3] = 0.5 if (vi+vj+vk%2==0) else 0.8
#print(colors[0,0,0])

# let's try setting the arrow widths -- hmm, similarly it does not seem possible to
# adjust the lengths or widths as a family.  They seem to be working on this
# https://github.com/matplotlib/matplotlib/issues/3382

# Hmm... plot.ly has no 3d quiver plot, either. 

# and neither plot.ly nor matplotlib have flowline support.  Hmm! 
# okay let's see if we can write code to rapidly generate flow lines. 

Before we move on to more sophisticated plots, let's do another example to demonstrate
how flexible our tools are.   I will compute the vector field around a [(p,q)-torus knot](../visualisation/plotly.pqtorus.ipynb), or alternatively see an [on-line demonstration](https://plot.ly/~Ryan.Budney/12/_72-torus-knot/). 

In [ ]:
import sympy as sp
t,x,y,z = sp.symbols('t x y z')
spp, spq, spr, spR = sp.symbols("p q r R", real=True)

c = sp.Matrix([(spR+spr*sp.cos(2*sp.pi*spq*t))*sp.cos(2*sp.pi*spp*t), 
     (spR+spr*sp.cos(2*sp.pi*spq*t))*sp.sin(2*sp.pi*spp*t), 
      spr*sp.sin(2*sp.pi*spq*t)])
cp = sp.diff(c, t) ## the derivative

In [ ]:
C = c.xreplace({spp: 3, spq: 2, spR: 1.2, spr: 0.4})
Cp = cp.xreplace({spp: 3, spq: 2, spR: 1.2, spr: 0.4})
P = sp.Matrix([x,y,z])

sp.pprint(Cp)

Recall our coordinate $\vec x$ in the integrand is the point on our curve.  

$$\frac{\frac{d \vec x}{dt} \times (\vec x - \vec p)}{|\vec x - \vec p|^3}dt$$

So in our case, we replace $\vec x$ with $c$ above, giving...

In [ ]:
num = Cp.cross(C-P)
num.simplify()
den = C - P
den = den.dot(den)**(3/2)
Fs = num/den
#sp.pprint(Fs)

F = []
for i in range(3):
    F.append(ufuncify([t,x,y,z], Fs[i,0]) )
    
Cc = [] ## the original curve
for i in range(3):
    Cc.append(ufuncify([t], C[i,0]))
    
x, y, z = np.mgrid[-1.5:1.5:5j, -1.5:1.5:5j, -0.5:0.5:3j]
u, v, w = quad_nvz3(F, [0, 2*np.pi], x,y,z )

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

T = np.arange(0.0, 2*np.pi, 0.01)

ax.plot(Cc[0](T), Cc[1](T), Cc[2](T), 'r')

q = ax.quiver(x, y, z, u, v, w, length=0.1, pivot='middle', cmap='Blues', linestyles='solid')
plt.show()

## quiver call details available in two locations.  The 3d plotting API
## http://matplotlib.org/mpl_toolkits/mplot3d/api.html
## and the linecollection API
## http://matplotlib.org/api/collections_api.html#matplotlib.collections.LineCollection

ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')

plt.show()

# Okay, we can compute magnetic fields in generality... but...

Our plotting capabilities are fairly limited.  Let's try to improve this by adding a basic flow line capability to our plots. 

In [ ]:
###### Basic flow lines.  We will take as input the initial vector field defined on the
## x,y,z meshgrid, and flow a small amount using an Euler method. 

## takes as input the same ingredients as before.  But for every x,y,z coordinate
## where we plot the vector fields, let's plot a small amount of the flow.  To 
## do this we will need to iterate over the x,y,z coordinates and perform a few
## steps of Euler's method. 

## function will return a list of flowlines.  An individual flowline will be
## a 3-element list [x,y,z] where x,y,z are lists of x-coordinates, y-coordinates,z-coordinates
## respectively for the individual flow line. 

def squad(F, I, *args):
    retval = []
    for f in F:
        retval.append(integrate.quad(f, I[0], I[1], args)[0])
    return retval

import itertools as it

x,y,z=np.mgrid[-1.4:1.4:8j, -1.4:1.4:8j, -0.8:0.8:6j]

## returns the flow lines as a (list) of (list of triples) of (list of floats)
## takes as input a mgrid of x,y,z coordinates, number of steps n and time step dt
## Also requires F callable function of 3-variables: t,x,y,z. We integrate against t,
## from 0 to 2pi for now. Might want to make this more flexible in the future.
## C is the original curve parametrization, assumed domain is [0,2pi]
def flowLines(x,y,z,F,C,n,dt):
    def onC(x,y,z,Cc):
        I = np.arange(0,2*np.pi, 0.01)
        if min([np.sqrt((Cc[0](t)-x)**2 + (Cc[1](t)-y)**2 + (Cc[2](t)-z)**2) for t in I]) < 0.02:
            return True ## close enough
        else:
            return False ## it's safe to use Euler's method
    
    retval = []
    for i,j,k in it.product(range(x.shape[0]), range(x.shape[1]), range(x.shape[2])):
        ## okay, let's build the flowline at x,y,z
        ## determine vector field at x,y,z, flow, build flowline, repeat
        xl = [x[i,j,k]]; yl = [y[i,j,k]]; zl = [z[i,j,k]];
        if onC(x[i,j,k], y[i,j,k], z[i,j,k], C) == False:
            for s in range(n):
                ## one step of Euler's method
                #integrate Fdt at xl,yl,zl from 0 to 2pi.
                #print(xl[-1], yl[-1], zl[-1])
                dp = squad(F, [0, 2*np.pi], xl[-1], yl[-1], zl[-1])
                ldp = np.sqrt(dp[0]**2+dp[1]**2+dp[2]**2)
                if (ldp<1.0): ldp = 1.0
                xl.append( xl[-1] + dt*dp[0]/ldp )
                yl.append( yl[-1] + dt*dp[1]/ldp )
                zl.append( zl[-1] + dt*dp[2]/ldp )
            
        ## append the flowline to retal.
        retval.append([xl, yl, zl])
        
    return retval

flows = flowLines(x,y,z, F, Cc, 70, 0.005)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

for P in flows:
    ax.plot(P[0], P[1], P[2], 'r')

T = np.arange(0.0, 2*np.pi, 0.01)

ax.plot(Cc[0](T), Cc[1](T), Cc[2](T), 'b')

plt.show()

### Let's see if Plot.ly does a better job

In [ ]:
## Let's see if we can do better with plot.ly
import plotly.offline as py
#import plotly.plotly as py
import plotly.graph_objs as go
#py.init_notebook_mode() # run at the start of every notebook

T = np.arange(0,2*np.pi, 0.004)

tracelist = [go.Scatter3d( ## the knot
    x=Cc[0](T),    y=Cc[1](T),     z=Cc[2](T),
    mode='lines',
    line=go.Line(color='#2050FF', width=18)
    )]

tipsx = [P[0][-1] for P in flows]
tipsy = [P[1][-1] for P in flows]
tipsz = [P[2][-1] for P in flows]

tracelist.append(go.Scatter3d(
    x=tipsx, y=tipsy, z=tipsz, mode='markers',
    marker = dict(
        size=3, color = 'yellow') ) )

for P in flows:
    tracelist.append( go.Scatter3d(x=P[0], y=P[1], z=P[2], 
                                   mode='lines',
                                   line=go.Line(color='#FF4040', width=5)) )

data=go.Data(tracelist)

layout = go.Layout(
    title='Vector Field around a conducting 3,2-torus knot',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='electric32torus.html')  #creates new page (large)
#py.iplot(fig) #puts inline
#py.plot(fig, filename='electric32torus.html') #puts online

okay, not bad.

## Let's try a current in a twisted sheet of metal

Let's begin with a paremetrization.  I'll choose the sheet to be a Moebius band. One way to write it down is:

$$(\theta, t) \longmapsto (\cos \theta, \sin \theta, 0) + 
t\left((\cos \theta, \sin \theta, 0)\cos(\frac{\theta}{2}) + (0,0,1)\sin(\frac{\theta}{2}) \right) $$

with $0 \leq \theta \leq 2\pi$ and $-1 \leq t \leq 1$, although one could choose the bounds for $t$ to be smaller if one likes. 

We will follow exactly the same formalism as for the previous examples.. . the exception being we will legistlate that the current is uniform and flowing precisely in the $\theta$ direction at a rate of one radian per second. 

In [ ]:
import sympy as sp
import numpy as np
from scipy import integrate
from sympy.utilities.autowrap import ufuncify

h,t,x,y,z = sp.symbols('h t x y z')

c = sp.Matrix([ sp.cos(h) + t*(sp.cos(h)*sp.cos(h/2)), sp.sin(h)+t*(sp.sin(h)*sp.cos(h/2)),
               t*sp.sin(h/2) ])
cp = sp.diff(c, t) ## the derivative
P = sp.Matrix([x,y,z])

sp.pprint(c)
## okay.

### Next

Make the integrand:

$$F(\vec x, \vec p) = \frac{d \vec I \times (\vec x - \vec p)}{|\vec x - \vec p|^3}$$

and then cast all the above into callable functions. 

In [ ]:
num = cp.cross(c-P)
num.simplify()
den = sp.Matrix(c-P)
den = den.dot(den)**(3/2)
Fs = num/den
sp.pprint(Fs)
## looks ok...

We have to integrate the vector field $Fs$ on the entire surface.  For this we need to appeal to the change-of-variables theorem.  Stated another way, we need to compute how the element of area in the $(\theta, t)$-plane is related to the element of area in the surface.  It is a general fact that if your surface sits in $R^n$ and your paremetrization is given by a function $F : R^k \to R^n$ then the element of $k$-dimensional volume in your parametric object is given by:

$$ \sqrt{ Det( DF^t \cdot DF ) } dx_1 dx_2 \cdots dx_k $$

In [ ]:
DF = sp.Matrix(3,2, lambda i,j: 0)
DF[:,0] = cp
DF[:,1] = sp.diff(c, h)
#sp.pprint(DF)
## the element of area:
EA = sp.sqrt( (DF.transpose()*DF).det() ).simplify()
sp.pprint(EA)

In [ ]:
## callable integrand
F = []
for i in range(3):
    F.append(ufuncify([h,t,x,y,z], Fs[i,0]) )
    
## callable surface
Cc = [] ## the original curve
for i in range(3):
    Cc.append(ufuncify([h,t], c[i,0]))
    
## callable d(surf)/dtheta
Cp = []
for i in range(3):
    Cp.append(ufuncify([h,t], cp[i,0]))
    
## callable F*EA - area distortion factor for the integration times F.
FEA = [EA*Fs[i,0] for i in range(3)]
Cfea = []
for i in range(3):
    Cfea.append( ufuncify([h,t,x,y,z], FEA[i] ) )

In [ ]:
x, y, z = np.mgrid[-1.5:1.5:10j, -1.5:1.5:10j, -0.5:0.5:8j]
## let's compute the initial vector field, and just plot that.. at first
u, v, w = nquad_nvz3(Cfea, [[0, 2*np.pi], [-0.4,0.4]], x,y,z )

In [ ]:
import matplotlib as mpl
%matplotlib nbagg
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import itertools as it

fig = plt.figure()
ax = fig.gca(projection='3d')

t1 = np.linspace(0, 2*np.pi, 200)
t2 = np.linspace(-0.4, 0.4, 30)

sx=np.empty( (len(t1), len(t2)) ); sy=np.empty( (len(t1), len(t2)) ); 
sz=np.empty( (len(t1), len(t2)) ); 
for i,j in it.product(range(len(t1)), range(len(t2))):
    sx[i,j]=Cc[0](t1[i], t2[j])
    sy[i,j]=Cc[1](t1[i], t2[j])
    sz[i,j]=Cc[2](t1[i], t2[j])

ax.set_title('Electified Moebius band', fontsize=14)
ax.plot_surface(sx, sy, sz, rstride=5, cstride=5, color='#FF2040', shade=True)

q = ax.quiver(x, y, z, u, v, w, length=0.1, pivot='middle', cmap='Blues', linestyles='solid')

ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')

plt.show()

## Again... kind of difficult to see

Let's try doing the same adaption to plot.ly


In [ ]:
## TODO: incomplete
def squad(F, I, *args):
    retval = []
    for f in F:
        retval.append(integrate.nquad(f, I, args)[0])
    return retval

x,y,z=np.mgrid[-1.4:1.4:8j, -1.4:1.4:8j, -0.8:0.8:6j]

## returns the flow lines as a (list) of (list of triples) of (list of floats)
## takes as input a mgrid of x,y,z coordinates, number of steps n and time step dt
## Also requires F callable function of 3-variables: t,x,y,z. We integrate against t,
## from 0 to 2pi for now. Might want to make this more flexible in the future.
## C is the original curve parametrization, assumed domain is [0,2pi]x[-0.4,0.4]
def flowLines(x,y,z,F,C,n,dt):
    def onC(x,y,z,Cc):
        I = np.arange(0,2*np.pi, 0.01)
        if min([np.sqrt((Cc[0](t)-x)**2 + (Cc[1](t)-y)**2 + (Cc[2](t)-z)**2) for t in I]) < 0.02:
            return True ## close enough
        else:
            return False ## it's safe to use Euler's method
    
    retval = []
    for i,j,k in it.product(range(x.shape[0]), range(x.shape[1]), range(x.shape[2])):
        ## okay, let's build the flowline at x,y,z
        ## determine vector field at x,y,z, flow, build flowline, repeat
        xl = [x[i,j,k]]; yl = [y[i,j,k]]; zl = [z[i,j,k]];
        if onC(x[i,j,k], y[i,j,k], z[i,j,k], C) == False:
            for s in range(n):
                ## one step of Euler's method
                #integrate Fdt at xl,yl,zl from 0 to 2pi.
                #print(xl[-1], yl[-1], zl[-1])
                dp = squad(F, [0, 2*np.pi], xl[-1], yl[-1], zl[-1])
                ldp = np.sqrt(dp[0]**2+dp[1]**2+dp[2]**2)
                if (ldp<1.0): ldp = 1.0
                xl.append( xl[-1] + dt*dp[0]/ldp )
                yl.append( yl[-1] + dt*dp[1]/ldp )
                zl.append( zl[-1] + dt*dp[2]/ldp )
            
        ## append the flowline to retal.
        retval.append([xl, yl, zl])
        
    return retval

flows = flowLines(x,y,z, F, Cc, 70, 0.005)